In [1]:
from pathlib import Path
from loguru import logger
import pickle
import numpy as np
from nltk.tokenize import WhitespaceTokenizer
from ipynb.fs.defs.Feature_Engineering import add_new_features
from ipynb.fs.defs.Feature_Engineering import preprocess_dataset
from ipynb.fs.full.Prepare_data import prepare_and_add_labels
from ipynb.fs.full.Prepare_data import load_csv_into_df
from ipynb.fs.full.Prepare_data import prepare_text

In [18]:
def main():
    # Define the path to the folder containing the data (xls files)
    data_path = Path("../data/original_data_new")
    dataframes = load_csv_into_df(data_path, original_prisma_data=False, move_to_archive=False)
    df, ncars = prepare_and_add_labels(dataframes, save_as_excel=False)
    
    # Load model
    model_path = "../models/lgbm_15052023_1541.pkl"
    with open(model_path, "rb") as fid:
        lgbm = pickle.load(fid)

    # Load the vectorizer from the file
    vectorizer_path = "../models/vectorizer_15052023_1541.pkl"
    with open(vectorizer_path, 'rb') as f:
        vectorizer = pickle.load(f)

    # Get the vocabulary of the training data
    vocab_path = '../models/vocabulary_15052023_1541.pkl'
    with open(vocab_path, 'rb') as f:
        vocabulary = pickle.load(f)
    
    for i in range(len(df)):

        df_preprocessed, df_for_plot = preprocess_dataset(df[i], cut_percent_of_front=0.25)

        logger.info(f"Start predicting relevant parts for the {ncars[i]} ...")

        df_preprocessed["Benennung (dt)"] = df_preprocessed.apply(lambda x: prepare_text(x["Benennung (dt)"]), axis=1)

        # Convert the vocabulary list to a dictionary
        vocabulary_dict = {word: index for index, word in enumerate(vocabulary)}

        # Set the vocabulary of the vectorizer to the loaded vocabulary
        vectorizer.vocabulary_ = vocabulary_dict
        X = vectorizer.transform(df_preprocessed['Benennung (dt)']).toarray()

        # Combine text features with other features
        #X = np.concatenate((X, df_preprocessed[['center_x', 'center_y', 'center_z','length','width','height','theta_x','theta_y','theta_z']].values), axis=1)

        y_pred = lgbm.predict(X)
        y_pred = np.round(y_pred)

        for index, row in df_preprocessed.iterrows():
            if y_pred[index] == 1: 
                df_preprocessed.loc[index,'Relevant fuer Messung'] = 'Ja'
            else:
                df_preprocessed.loc[index,'Relevant fuer Messung'] = 'Nein'

        df_preprocessed = df_preprocessed[df_preprocessed['Relevant fuer Messung'] == 'Ja']

        df_preprocessed = df_preprocessed.loc[:,["Sachnummer", "Benennung (dt)", "Relevant fuer Messung", "Einheitsname"]]

        df_preprocessed.to_excel(f"../data/predicted/{ncars[i]}_relevante_Bauteile.xlsx")

        logger.success(f"The prediction is done and the result is stored here: data/predicted/{ncars[i]}_labeled_test.xlsx!")

        logger.info('__________________________________________________________________________________________')

In [19]:
if __name__ == "__main__":
    
    main()

2023-05-15 16:26:27.312 | INFO     | ipynb.fs.full.Prepare_data:load_csv_into_df:34 - Loading the data...
2023-05-15 16:26:34.344 | SUCCESS  | ipynb.fs.full.Prepare_data:load_csv_into_df:68 - 4 dataframe(s) were created.
2023-05-15 16:26:34.345 | INFO     | ipynb.fs.full.Prepare_data:prepare_and_add_labels:106 - Start preprocessing the data...
2023-05-15 16:26:34.624 | SUCCESS  | ipynb.fs.full.Prepare_data:prepare_and_add_labels:184 - The features are reduced and formated to the correct data type!
2023-05-15 16:26:34.667 | INFO     | ipynb.fs.defs.Feature_Engineering:preprocess_dataset:199 - Start preprocessing the dataframe with 2394 samples...
2023-05-15 16:26:36.777 | SUCCESS  | ipynb.fs.defs.Feature_Engineering:preprocess_dataset:255 - The dataset is successfully preprocessed. The new dataset contains 469 samples
2023-05-15 16:26:36.778 | INFO     | __main__:main:26 - Start predicting relevant parts for the NA7 ...
2023-05-15 16:26:37.155 | SUCCESS  | __main__:main:55 - The predict